In [1]:
import pandas as pd

In [2]:
matches_df = pd.read_csv("~/storage/data/physicists/forks-of-cmssw-2022-03-04/GitHub-CMSSW-user-nonfork-dependencies-2022-03-04.csv", parse_dates=["date"])
matches_df

,owner,repo,date,filename,suffix,package,how
0,helee,helee/HN_Plotter,2020-03-11 08:11:46,helee-HN_Plotter/makePlots_muon_eachYear.C,C,iostream,cpp_include_bracket
1,helee,helee/HN_Plotter,2020-03-11 08:11:46,helee-HN_Plotter/makePlots_TT_eachYear.C,C,iostream,cpp_include_bracket
2,helee,helee/HN_Plotter,2020-03-11 08:11:46,helee-HN_Plotter/makePlots_DY_eachYear.C,C,iostream,cpp_include_bracket
3,helee,helee/HN_Plotter,2020-03-11 08:11:46,helee-HN_Plotter/makePlots_electron_eachYear.C,C,iostream,cpp_include_bracket
4,helee,helee/TnP_analysis,2018-10-23 06:25:20,helee-TnP_analysis/treeAnalysis.C,C,TH2.h,cpp_include_bracket
...,...,...,...,...,...,...,...
5224784,hkaushalya,hkaushalya/haddws,2014-01-21 23:24:01,hkaushalya-haddws/haddws.C,C,TH1.h,cpp_include_quote
5224785,hkaushalya,hkaushalya/haddws,2014-01-21 23:24:01,hkaushalya-haddws/haddws.C,C,TTree.h,cpp_include_quote
5224786,hkaushalya,hkaushalya/haddws,2014-01-21 23:24:01,hkaushalya-haddws/haddws.C,C,TKey.h,cpp_include_quote
5224787,hkaushalya,hkaushalya/haddws,2014-01-21 23:24:01,hkaushalya-haddws/haddws.C,C,Riostream.h,cpp_include_quote


### Top file suffixes

In [19]:
matches_df[["filename", "suffix", "how"]].groupby("filename").first().groupby("suffix").count()["how"].sort_values(ascending=False).head(16)

suffix
py       327740
h        168174
cc       120165
C         65537
cpp       48980
c         34114
cxx       28209
hpp       27206
hh        19873
ipynb      8303
H           667
c++          73
hh~          20
json         15
CPP          10
CC            9
Name: how, dtype: int64

### Identify language

In [38]:
is_python = (matches_df["suffix"] == "py") & ((matches_df["how"] == "python_import") | (matches_df["how"] == "python_fromimport"))

In [39]:
is_python_jupyter = (matches_df["suffix"] == "ipynb") & ((matches_df["how"] == "python_import") | (matches_df["how"] == "python_fromimport"))

In [40]:
is_cpp = (matches_df["suffix"].isin(["h", "cc", "C", "cpp", "c", "cxx", "hpp", "hh", "H", "c++", "CPP", "CC"])) & \
         ((matches_df["how"] == "cpp_include_bracket") | (matches_df["how"] == "cpp_include_quote"))

In [50]:
is_cpp_jupyter = (matches_df["suffix"] == "ipynb") & ((matches_df["how"] == "cpp_include_bracket") | (matches_df["how"] == "ipynb_include_quotedquote"))

### Cut against framework Python files

In [67]:
by_filename = matches_df.set_index("filename")

In [73]:
any_FWCore = (by_filename["package"] == "FWCore").groupby(level=0).any()

In [96]:
is_framework = any_FWCore.reindex(by_filename.index).values

### How many Python/Jupyter/C++ files

In [105]:
matches_df[is_python & is_framework].groupby("filename").count()["how"].sort_values(ascending=False)

filename
samhiggie-PUAnalysis/Condor/SUB_Data.py                                                                        69
saxenapooja-HTauTau/desy_setup/CMSSW_5_3_9_patch3/python/RecoTauTag/Configuration/updateHPSPFTaus_cff.py       67
tmrhombus-UWAnalysis/Wbb8TeV/farmoutAJ_analyzer/text/python/RecoTauTag/Configuration/updateHPSPFTaus_cff.py    67
scooperstein-UWHiggsZHBackup/python/RecoTauTag/Configuration/updateHPSPFTaus_cff.py                            67
rmanzoni-HTT/RecoTauTag/Configuration/python/updateHPSPFTaus_cff.py                                            65
                                                                                                               ..
ku-cms-TprimeAna/python/QCDHT2000v2-2_cfi.py                                                                    1
ku-cms-TprimeAna/python/QCDHT2000v2-1_cfi.py                                                                    1
ku-cms-TprimeAna/python/QCDHT2000_cfi.py                                       

In [106]:
matches_df[is_python & ~is_framework].groupby("filename").count()["how"].sort_values(ascending=False)

filename
Dr15Jones-CID-App/PythonistaAppTemplate/PythonistaKit.framework/pylib_ext/sympy/polys/polytools.py        571
PerilousApricot-CRAB2/external/DBSAPI/DBSAPI/dbsApi.py                                                    230
tklijnsma-Archive/MEAnalysis/gc/grid-control/packages/grid_control_cms/DBSAPI/dbsApi_DEPRECATED.py        228
Dr15Jones-CID-App/PythonistaAppTemplate/PythonistaKit.framework/pylib_ext/sympy/polys/compatibility.py    207
Dr15Jones-CID-App/PythonistaAppTemplate/PythonistaKit.framework/pylib_ext/sympy/matrices/matrices.py      189
                                                                                                         ... 
sergojin-L1TMuonSimulations/Analyzers/test6/utils.py                                                        1
cms-tau-pog-TauPerformanceNANO/cmsstyle.py                                                                  1
cms-tau-pog-TauReleaseValidation/Var.py                                                                     1
s

In [108]:
# importing FWCore in a Jupyter Notebook would be a strange thing to do...
matches_df[is_python_jupyter & is_framework].groupby("filename").count()["how"].sort_values(ascending=False)

filename
NJManganelli-FourTopNAOD/test/tagVarAdd.ipynb                                 7
cms-btv-pog-CMSPOS-BTaggingExercise/notebooks/Exercise2_reclustering.ipynb    7
cms-btv-pog-CMSPOS-BTaggingExercise/notebooks/Exercise2.ipynb                 6
Name: how, dtype: int64

In [109]:
matches_df[is_python_jupyter & ~is_framework].groupby("filename").count()["how"].sort_values(ascending=False)

filename
ocerri-BPH_RD_Analysis/Combine/deprecated/RD-MC_side_v0.ipynb                                                                               3228
ocerri-BPH_RD_Analysis/Combine/deprecated/CreateCombineCard_v1.ipynb                                                                        1179
thaarres-gof_tests/gof.ipynb                                                                                                                 319
kingusiu-cofitan/Simple_combine_na3.5TeV_s5_q90.ipynb                                                                                        313
kingusiu-cofitan/Simple_combine_na3.5TeV_s5_q30.ipynb                                                                                        313
                                                                                                                                            ... 
mattbellis-air_monitoring/python/sample_calculations.ipynb                                                               

In [110]:
matches_df[is_cpp].groupby("filename").count()["how"].sort_values(ascending=False)

filename
VinInn-ctest/stringLess/Configuration.cc                                                                                                         2000
VinInn-ctest/stringLess/ConfigurationOld.cc                                                                                                      2000
caleb-james-smith-LearningTensorflow/rnn/data/kernel.c                                                                                            902
pjurgielewicz-ROOT_LITE_6/test/stressRooFit_tests.cxx                                                                                             563
pjurgielewicz-ROOT_LITE/test/stressRooFit_tests.cxx                                                                                               563
                                                                                                                                                 ... 
giorgiopizz-progetti_elettronica/arm_progetto_1/I2C_display_termometro/Drivers/CMSIS/DSP/So

In [111]:
matches_df[is_cpp_jupyter].groupby("filename").count()["how"].sort_values(ascending=False)

filename
xiezhen-notebooks/brilsimulator.ipynb                               78
xiezhen-notebooks/brildata.ipynb                                    64
xiezhen-notebooks/brilmonitoring.ipynb                              35
ocerri-PID_timing_studies/plot_script/Delphes_tree_studies.ipynb    33
hevjinyarar-ranbox/ranbox22-MODIFIED.ipynb                          31
                                                                    ..
shchablo-CRO/jump/archive/MUX.ipynb                                  1
shchablo-CRO/jump/archive/MUX_1170.ipynb                             1
shchablo-CRO/jump/archive/ev2plot.ipynb                              1
shchablo-CRO/jump/archive/ped2overview.ipynb                         1
shchablo-CRO/jump/archive/13-05-21_MUX_doCode.ipynb                  1
Name: how, Length: 86, dtype: int64

### Top Python packages (in files _without_ FWCore)

In [101]:
matches_df[(is_python | is_python_jupyter) & ~is_framework].groupby("package").count()["how"].sort_values(ascending=False).head(50)

package
ROOT               59681
os                 57313
sys                40646
numpy              29545
matplotlib         28224
array              21573
math               21382
__future__         16585
re                 14113
time               11685
argparse           11557
optparse           11384
HiggsAnalysis       9854
pandas              9349
madgraph            9248
glob                9187
OptionParser        8617
subprocess          7816
json                7797
logging             7682
Configuration       7598
copy                7578
of                  6954
datetime            6833
config              6503
print_function      6350
collections         5509
division            5056
random              4698
itertools           4435
pickle              4363
absolute_import     4351
tensorflow          4253
shutil              3977
keras               3797
internal            3544
pprint              3420
scipy               3329
multiprocessing     3124
TFile            

### Top C++ packages

In [ ]:
matches_df[(is_cpp | is_cpp_jupyter)].groupby("package").count()["how"].sort_values(ascending=False).head(50)

package
iostream                                          108647
vector                                             85492
string                                             73077
TFile.h                                            64713
TCanvas.h                                          42748
TTree.h                                            38961
TROOT.h                                            37035
TMath.h                                            36096
fstream                                            36019
FWCore/ParameterSet/interface/ParameterSet.h       31641
sstream                                            29742
FWCore/Framework/interface/Event.h                 29383
TStyle.h                                           28689
map                                                28513
TString.h                                          28122
FWCore/Framework/interface/MakerMacros.h           27985
TH1.h                                              26767
memory                 